In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [66]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Initialize ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)  # Only normalization for validation

# Create data generators
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/videos_deep/train',  # Path to training data
    target_size=(224, 224),  # Image size
    batch_size=32,
    class_mode='binary'  # Change to 'categorical' for multi-class classification
)

validation_generator = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/videos_deep/validation',  # Path to validation data
    target_size=(224, 224),  # Image size
    batch_size=32,
    class_mode='binary'  # Change to 'categorical' for multi-class classification
)


Found 425 images belonging to 2 classes.
Found 1440 images belonging to 2 classes.


In [67]:
# Initialize ResNet50 base model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)  # Binary classification (real vs fake)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model to prevent training on them
for layer in base_model.layers:
    layer.trainable = False


In [68]:
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [69]:
print("Training data:", train_generator.n)
print("Validation data:", validation_generator.n)


Training data: 425
Validation data: 1440


In [75]:
print( validation_generator.batch_size)

32


In [78]:
history = model.fit(
    train_generator,
    batch_size=32,
    epochs=5,  # Adjust the number of epochs as needed
    validation_data=validation_generator,

    )


Epoch 1/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 13s 786ms/step - accuracy: 0.5709 - loss: 0.6739 - val_accuracy: 0.8333 - val_loss: 0.5885
Epoch 2/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 13s 752ms/step - accuracy: 0.5852 - loss: 0.6688 - val_accuracy: 0.8333 - val_loss: 0.5860
Epoch 3/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 742ms/step - accuracy: 0.5900 - loss: 0.6693 - val_accuracy: 0.8236 - val_loss: 0.6416
Epoch 4/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 20s 735ms/step - accuracy: 0.5897 - loss: 0.6697 - val_accuracy: 0.8333 - val_loss: 0.5976
Epoch 5/5
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 719ms/step - accuracy: 0.4853 - loss: 0.7130 - val_accuracy: 0.8333 - val_loss: 0.4652


In [79]:
# If you have a separate test set
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/videos_deep/test',  # Path to test data
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_loss, test_accuracy = model.evaluate(
    test_generator,
    steps=test_generator.samples // test_generator.batch_size
)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Found 277 images belonging to 2 classes.


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.7245 - loss: 0.5906
Test Loss: 0.5766342282295227
Test Accuracy: 0.734375


In [83]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_deepfake(model, img_path, target_size=(224, 224)):
    """
    Predicts the probability that the input image is a deepfake or real using a trained model.

    Args:
        model (tf.keras.Model): Trained model.
        img_path (str): Path to the input image.
        target_size (tuple): Target size to which the image will be resized.

    Returns:
        float: Probability of the image being classified as 'real'.
    """
    # Load and preprocess the image
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to [0, 1] range

    # Make prediction
    predictions = model.predict(img_array)

    # The prediction output is the probability of the positive class (real)
    probability_real = predictions[0][0]  # Probability that the image is 'real'

    return probability_real

# Example usage
img_path = '/content/drive/MyDrive/videos_deep/test/fake/067_025_10.png'
probability_real = predict_deepfake(model, img_path)
print(f"Probability that the image is real: {probability_real:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Probability that the image is real: 0.7515
